In [1]:
import pandas as pd
import scanpy as sc
import scConnect as cn
import matplotlib
import matplotlib.pyplot as plt

In [2]:
adata = sc.read_csv("./input/CID4465_count.csv").T
adata.X.shape

(1554, 16880)

In [3]:
meta = pd.read_csv("./input/CID4465_metadata.csv",  index_col="Cell")
adata.obs = meta
adata.obs.head(10)

,Annotation
Cell,
CID4465_AAACGGGTCATCGATG,Endothelial
CID4465_AAACGGGTCGGAAACG,Endothelial
CID4465_AAAGATGAGGTGTGGT,Endothelial
CID4465_AAAGCAATCCTTTACA,Endothelial
CID4465_AACCATGGTACGCTGC,Endothelial
CID4465_AACGTTGGTACGAAAT,Endothelial
CID4465_AACTCAGCATTGTGCA,Endothelial
CID4465_AACTCCCGTAGCTGCC,Endothelial
CID4465_AACTGGTTCGGTGTCG,Endothelial


In [4]:
adata_tissue = cn.genecall.meanExpression(adata, groupby="Annotation", normalization=False, use_raw=False, transformation="log1p")

In [5]:
cn.database.version = "2020-5"

In [6]:
pd.DataFrame(adata_tissue.uns["gene_call"])

,B-cells,CAFs,Cancer Epithelial,Endothelial,Myeloid,PVL,Plasmablasts,T-cells
FO538757.2,0.219428,0.469855,0.705697,0.368467,0.649046,0.581656,0.040332,0.308985
AP006222.2,0.000000,0.295797,0.338537,1.117015,0.207648,0.781015,0.042402,0.000000
RP4-669L17.10,1.021037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RP5-857K21.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.059746,0.000000,0.000000
RP11-206L10.9,0.046392,0.050018,0.137401,0.049655,0.000000,0.143049,0.000000,0.248435
...,...,...,...,...,...,...,...,...
RP11-352D3.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182256
RP1-310O13.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RP11-80K6.2,0.000000,0.000000,0.000000,0.083984,0.000000,0.000000,0.000000,0.000000
ZSCAN4,0.000000,0.000000,0.000000,0.004642,0.000000,0.000000,0.000000,0.000000


In [8]:
adata_tissue = cn.connect.ligands(adata_tissue, organism="hsapiens")

/home/ljx/miniconda3/envs/cell2cell/lib/python3.7/site-packages/scipy/stats/stats.py:275: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [9]:
adata_tissue = cn.connect.receptors(adata_tissue, organism="hsapiens")

In [10]:
adata_tissue = cn.connect.specificity(adata_tissue, n=100, groupby="Annotation", organism="hsapiens")

Shuffeling dataframe 100 out of 100 |██████████████████████████████| 100.0% 
Calculating means...
Calculating standard deviations...
Calculating Z-score, p-values and corrected p-values...
45.91404358353511 % of group metrices were 0. increase n to reduce this number
29.142809026608283 % of group metrices were 0. increase n to reduce this number


In [11]:
edges = cn.connect.interactions(emitter=adata_tissue, target=adata_tissue, self_reference=True, organism="hsapiens")
nodes = cn.connect.nodes(adata_tissue)

finding connections between 8 emitter clusters and 8 target clusters |██████████████████████████████| 100.0% 
precessing adata #1
processing cluster B-cells
processing cluster CAFs
processing cluster Cancer Epithelial
processing cluster Endothelial
processing cluster Myeloid
processing cluster PVL
processing cluster Plasmablasts
processing cluster T-cells


In [12]:
edge = pd.DataFrame(edges)
edge

,0,1,2
0,B-cells,B-cells,"{'score': 0.08253783210578249, 'log_score': 0...."
1,B-cells,B-cells,"{'score': 0.40702988662237255, 'log_score': 0...."
2,B-cells,B-cells,"{'score': 0.08027870278185381, 'log_score': 0...."
3,B-cells,B-cells,"{'score': 0.24499916505626873, 'log_score': 0...."
4,B-cells,B-cells,"{'score': 0.39996108511463474, 'log_score': 0...."
...,...,...,...
2561,T-cells,T-cells,"{'score': 0.23624535637466676, 'log_score': 0...."
2562,T-cells,T-cells,"{'score': 0.23624535637466676, 'log_score': 0...."
2563,T-cells,T-cells,"{'score': 0.21490104611385633, 'log_score': 0...."
2564,T-cells,T-cells,"{'score': 0.22041675105764508, 'log_score': 0...."


In [13]:
col_new = list(edge.iloc[0,2].keys())
col_new

['score',
 'log_score',
 'ligand',
 'ligand_zscore',
 'ligand_pval',
 'receptor',
 'receptor_zscore',
 'receptor_pval',
 'interaction',
 'specificity',
 'importance',
 'endogenous',
 'action',
 'ligandspecies',
 'receptorspecies',
 'pubmedid',
 'receptorfamily',
 'receptorgene',
 'ligandtype',
 'ligandcomment']

In [14]:
edge.rename(columns={0:'sender', 1:'reciever', 2:'info'}, inplace = True)
edge

,sender,reciever,info
0,B-cells,B-cells,"{'score': 0.08253783210578249, 'log_score': 0...."
1,B-cells,B-cells,"{'score': 0.40702988662237255, 'log_score': 0...."
2,B-cells,B-cells,"{'score': 0.08027870278185381, 'log_score': 0...."
3,B-cells,B-cells,"{'score': 0.24499916505626873, 'log_score': 0...."
4,B-cells,B-cells,"{'score': 0.39996108511463474, 'log_score': 0...."
...,...,...,...
2561,T-cells,T-cells,"{'score': 0.23624535637466676, 'log_score': 0...."
2562,T-cells,T-cells,"{'score': 0.23624535637466676, 'log_score': 0...."
2563,T-cells,T-cells,"{'score': 0.21490104611385633, 'log_score': 0...."
2564,T-cells,T-cells,"{'score': 0.22041675105764508, 'log_score': 0...."


In [15]:
for each in col_new:
    edge[each] = edge["info"].map(lambda x:x[each])

In [16]:
edge.to_csv("./result/output.scConnect.csv")

In [17]:
G_tissue = cn.graph.build_graph(edges, nodes)

Graph has 2566 interactions between 8 clusters
B-cells has 364 interactions
CAFs has 746 interactions
Cancer Epithelial has 553 interactions
Endothelial has 814 interactions
Myeloid has 795 interactions
PVL has 668 interactions
Plasmablasts has 655 interactions
T-cells has 537 interactions
